<a href="https://colab.research.google.com/github/0rodrigo/CompIME/blob/master/solvimm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Desafio Estágio em Dados - Solvimm

## Foi utilizado o Google Colab
https://colab.research.google.com/

Antes de executar basta fazer o upload dos datasets movies.csv e customers_rating.csv

In [113]:
import pandas as pd

##### Leitura do arquivo movies.csv

In [114]:
# Lendo o arquivo e colocando o header
df_movies = pd.read_csv('../content/movies.csv', sep = ';',names=['Movie_Id','Movie_Name'], usecols=[0,1])

# Tirando os parentêses
df_movies["Movie_Name"] = df_movies["Movie_Name"].map(lambda x: str(x)[1:-1])

# Separando o titulo e o ano de lançamento
new = df_movies["Movie_Name"].str.split(",", n = 1, expand = True)

# Alterando o titulo e acrescentando a coluna do ano de lançamento
df_movies["Movie_Name"] = new[0]
df_movies["Year_Released"] = new[1]

##### Leitura do arquivo customers_rating.csv

In [115]:
df_customers = pd.read_csv('../content/customers_rating.csv', sep = ';')

##### Fazendo o merge dos dois dataframes

In [116]:
df_merged = pd.merge(df_movies, df_customers, left_on='Movie_Id', right_on='Movie_Id')

##### 1.1 Quantos filmes estão disponíveis no dataset?

In [117]:
df_merged['Movie_Id'].nunique()

4499

##### 1.2 Qual é o nome dos 5 filmes com melhor média de avaliação?

In [118]:
# Calculando a média de cada filme
df_rating = df_merged.groupby(['Movie_Name'])['Rating'].mean()

# Ordenando o pelo valor das médias
df_rating = df_rating.sort_values(ascending=False)

# Selecionando os 5 filmes mais bem avaliados
df_rating.head(5)

Movie_Name
Lost: Season 1                                                                 4.670989
Ghost in the Shell: Stand Alone Complex: 2nd Gig                               4.586364
The Simpsons: Season 6                                                         4.581296
Inu-Yasha                                                                      4.554434
Lord of the Rings: The Return of the King: Extended Edition: Bonus Material    4.552000
Name: Rating, dtype: float64

##### 1.3 Quais os 5 anos com menos lançamentos de filmes?

In [119]:
# Contando o numero de filmes de cada ano
Year=df_movies['Year_Released'].value_counts(ascending=True)
Year.head(5)

 1917    1
 1926    1
 1922    1
 1915    1
 1916    2
Name: Year_Released, dtype: int64

##### 1.4 Quantos filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data de avaliação do dataset?

In [120]:
# Ordenando pela data e escolhendo a ultima data do dataset:
df_date = df_merged.sort_values(by='Date',ascending=False)
df_date = df_date['Date']
last_date = pd.merge(df_merged, df_date.head(1),left_on='Date', right_on='Date')

# Calulando a média apenas para os filmes avaliados na última data e ordenando pela média:
df_rating = last_date.groupby(['Movie_Id'], as_index=False)['Rating'].mean().sort_values(by='Rating', ascending=False)

# Selecionando apenas os filmes que possuem media maior ou igual a 4,7:
df_date_rating = df_rating.loc[df_rating['Rating']>=4.7]
df_date_rating['Movie_Id'].nunique()

195

##### 1.5 Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e quais as notas?

In [121]:
# Realizando a combinação do Movie_Id dos filmes filtrados na questao anterior com o dataset que contem os nomes para caputar o nome dos filmes
worst_movies = pd.merge(df_date_rating,df_movies, left_on='Movie_Id', right_on='Movie_Id').drop(columns=['Movie_Id','Year_Released'])

# Ordenando em ordem decrescente os filmes disponíveis:
worst_movies = worst_movies.sort_values(by='Rating', ascending=True)

worst_movies.head(10)

,Rating,Movie_Name
194,4.714286,Spirited Away
187,4.750000,Pete's Dragon
188,4.750000,SpongeBob SquarePants: Season 3
189,4.750000,Gilmore Girls: Season 3
193,4.750000,Absolutely Fabulous: Series 2
191,4.750000,The Twilight Zone: Vol. 15
192,4.750000,An Evening With Kevin Smith
190,4.750000,The Twilight Zone: Vol. 41
186,4.800000,Curb Your Enthusiasm: Season 3
185,4.800000,In the Mood for Love


##### 1.6 Quais os id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um fez?

In [122]:
# Calculando a quantidade de avaliações feitas por cada Cust_Id
customers = df_merged.groupby(by='Cust_Id')['Rating'].count()

# Apresentando o top 5 dos customers que mais fizeram avaliações
customers.sort_values(ascending=False).head(5)

Cust_Id
305344     4467
387418     4422
2439493    4195
1664010    4019
2118461    3769
Name: Rating, dtype: int64